In [ ]:
import pandas as pd
import numpy as np
import cv2, os, shutil, math
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score, classification_report, confusion_matrix
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
import os
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def make_dataframes(sdir):
    bad_images = []  # Giữ lại nếu có ảnh bị lỗi, nhưng không cần xử lý
    filepaths = []
    labels = []
    classes = sorted(os.listdir(sdir))  # Lấy danh sách các lớp bệnh (thư mục con)
    
    # Duyệt qua các thư mục bệnh
    for klass in classes:
        classpath = os.path.join(sdir, klass)
        if not os.path.isdir(classpath):  # Nếu không phải thư mục thì bỏ qua
            continue
        flist = sorted(os.listdir(classpath))
        desc = f'{klass:23s}'
        
        # Duyệt qua từng tệp trong thư mục
        for f in tqdm(flist, ncols=110, desc=desc, unit='file', colour='blue'):
            fpath = os.path.join(classpath, f)
            try:
                # Đọc ảnh mà không cần kiểm tra phần mở rộng
                img = cv2.imread(fpath)
                shape = img.shape  # Lấy kích thước ảnh
                filepaths.append(fpath)
                labels.append(klass)
            except Exception as e:
                bad_images.append(fpath)  # Lưu lại ảnh bị lỗi
                print(f'Defective image file: {fpath}, Error: {e}')
    
    # Tạo DataFrame từ danh sách filepaths và labels
    Fseries = pd.Series(filepaths, name='filepaths')
    Lseries = pd.Series(labels, name='labels')
    df = pd.concat([Fseries, Lseries], axis=1)
    
    # Chia dữ liệu thành train, validation và test (80-10-10)
    train_df, dummy_df = train_test_split(df, train_size=.8, shuffle=True, random_state=123, stratify=df['labels'])
    valid_df, test_df = train_test_split(dummy_df, train_size=.5, shuffle=True, random_state=123, stratify=dummy_df['labels'])
    
    # Tính toán một số thông số về dữ liệu
    classes = sorted(train_df['labels'].unique())
    class_count = len(classes)
    sample_df = train_df.sample(n=50, replace=False)
    
    ht, wt, count = 0, 0, 0
    for i in range(len(sample_df)):
        fpath = sample_df['filepaths'].iloc[i]
        try:
            img = cv2.imread(fpath)
            h, w = img.shape[:2]
            ht += h
            wt += w
            count += 1
        except:
            pass
    if count > 0:
        ave_height = ht // count
        ave_width = wt // count
        aspect_ratio = ave_height / ave_width
    else:
        ave_height, ave_width, aspect_ratio = 0, 0, 0
    
    # Hiển thị thông tin thống kê
    print(f'Number of classes in processed dataset: {class_count}')
    counts = list(train_df['labels'].value_counts())
    print(f'Max files in any class in train_df: {max(counts)}, Min files in any class: {min(counts)}')
    print(f'Train dataset length: {len(train_df)}, Test dataset length: {len(test_df)}, Validation dataset length: {len(valid_df)}')
    print(f'Average image height: {ave_height}, Average image width: {ave_width}, Aspect ratio (height/width): {aspect_ratio}')
    
    return train_df, test_df, valid_df, classes, class_count

# Đọc và chia dữ liệu
sdir = 'D:/2011/folder1'  # Đường dẫn tới thư mục ảnh
train_df, test_df, valid_df, classes, class_count = make_dataframes(sdir)


In [ ]:
n=2500
batch_size = 32
working_dir=r'./'
img_size=(300, 300)
epochs = 50
input_shape = (300, 300, 3)


def balance(df, n, working_dir, img_size):
    df = df.copy()
    print('Initial length of dataframe is ', len(df))
    aug_dir = os.path.join(working_dir, 'aug')
    if os.path.isdir(aug_dir):
        shutil.rmtree(aug_dir)
    os.mkdir(aug_dir)

    for label in df['labels'].unique():
        dir_path = os.path.join(aug_dir, label)
        os.mkdir(dir_path)
    total = 0
    gen = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,  # Thêm shear
    fill_mode='reflect',  # Thay đổi từ 'nearest' sang 'reflect'
    brightness_range=[0.8, 1.2],  # Thêm brightness augmentation
    channel_shift_range=50.0,  # Thêm channel shift
    validation_split=0.2
    )
    groups = df.groupby('labels')
    for label in df['labels'].unique():
        group = groups.get_group(label)
        sample_count = len(group)
        if sample_count < n:
            aug_img_count = 0
            delta = n - sample_count
            target_dir = os.path.join(aug_dir, label)
            msg = '{0:40s} for class {1:^30s} creating {2:^5s} augmented images'.format(' ', label, str(delta))
            print(msg, '\r', end='')  # prints over on the same line
            aug_gen = gen.flow_from_dataframe(group, x_col='filepaths', y_col=None, target_size=img_size,
                                              class_mode=None, batch_size=batch_size, shuffle=False,
                                              save_to_dir=target_dir, save_prefix='aug-', color_mode='rgb',
                                              save_format='jpg')
            while aug_img_count < delta:
                images = next(aug_gen)
                aug_img_count += len(images)
            total += aug_img_count
    print('Total Augmented images created= ', total)
    aug_fpaths, aug_labels = [], []
    classlist = os.listdir(aug_dir)
    for target in classlist:
        classpath = os.path.join(aug_dir, target)
        flist = os.listdir(classpath)
        for f in flist:
            fpath = os.path.join(classpath, f)
            aug_fpaths.append(fpath)
            aug_labels.append(target)
    Fseries = pd.Series(aug_fpaths, name='filepaths')
    Lseries = pd.Series(aug_labels, name='labels')
    aug_df = pd.concat([Fseries, Lseries], axis=1)
    df = pd.concat([df, aug_df], axis=0).reset_index(drop=True)
    print('Length of augmented dataframe is ', len(df))
    return df


train_df = balance(train_df, n, working_dir, img_size)

In [ ]:
def make_gens(batch_size, train_df, test_df, valid_df, img_size):
    trgen = ImageDataGenerator(horizontal_flip=True)
    t_and_v_gen = ImageDataGenerator()
    msg = '{0:70s} for train generator'.format(' ')
    print(msg, '\r', end='')
    train_ds = trgen.flow_from_dataframe(train_df, x_col='filepaths', y_col='labels',
                                         target_size=img_size, class_mode='categorical',
                                         color_mode='rgb', batch_size=batch_size, shuffle=True)

    msg = '{0:70s} for valid generator'.format(' ')
    print(msg, '\r', end='')
    valid_ds = t_and_v_gen.flow_from_dataframe(valid_df, x_col='filepaths', y_col='labels',
                                         target_size=img_size, class_mode='categorical',
                                         color_mode='rgb', batch_size=batch_size, shuffle=False)

    test_len = len(test_df)
    test_batch_size = sorted([int(test_len / n) for n in range(1, test_len + 1)
                              if test_len % n == 0 and test_len / n<=80], reverse=True)[0]
    test_steps = int(test_len / test_batch_size)
    msg = '{0:70s} for test generator'.format(' ')
    print(msg, '\r', end='')
    test_ds = t_and_v_gen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels',
                                               target_size=img_size, class_mode='categorical',
                                               color_mode='rgb', batch_size=batch_size, shuffle=False)

    classes = list(train_ds.class_indices.keys())
    class_count = len(classes)
    print('test batch size: ', test_batch_size, 'test steps: ', test_steps, 'number of classes : ', class_count)

    return train_ds, test_ds, valid_ds

train_ds, test_ds, valid_ds = make_gens(batch_size, train_df, test_df, valid_df, img_size)

In [ ]:
# 1. Tải InceptionV3 pre-trained model
base_model_inception = InceptionV3(
    weights="imagenet",
    input_shape=input_shape, 
    include_top=False
)

# 2. Đóng băng một phần các lớp của base_model
# Chỉ mở khóa các lớp cuối để fine-tuning
for layer in base_model_inception.layers[:-50]:  # Giữ đóng băng các lớp đầu
    layer.trainable = False
for layer in base_model_inception.layers[-50:]:  # Mở khóa các lớp cuối để fine-tuning
    layer.trainable = True

# 3. Thêm các lớp tùy chỉnh với kiến trúc mạnh hơn
x_inception = GlobalAveragePooling2D(name='inception_avg_pool')(base_model_inception.output)
x_inception = BatchNormalization()(x_inception)
x_inception = Dense(1024, activation="relu", name='inception_fc1')(x_inception)
x_inception = BatchNormalization()(x_inception)
x_inception = Dropout(0.5, name='inception_dropout1')(x_inception)  # Tăng dropout để giảm overfitting
x_inception = Dense(512, activation="relu", name='inception_fc2')(x_inception)
x_inception = BatchNormalization()(x_inception)
x_inception = Dropout(0.3, name='inception_dropout2')(x_inception)
prediction_inception = Dense(class_count, activation='softmax', name='inception_predictions')(x_inception)

# 4. Tạo model InceptionV3 hoàn chỉnh
inception_model = Model(inputs=base_model_inception.input, outputs=prediction_inception, name='inception_custom')

# 5. Callback để giảm learning rate và early stopping
learning_rate_reduction_inception = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=3,
    verbose=1,
    factor=0.5,
    min_lr=0.000001
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    verbose=1,
    restore_best_weights=True
)

# 6. Compile model với optimizer tốt hơn
inception_model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Sử dụng Adam thay vì RMSprop
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

# 7. In summary của model InceptionV3
print("\nCấu trúc mô hình InceptionV3:")
inception_model.summary()


# 9. Huấn luyện model InceptionV3 với nhiều epochs hơn
print("\nBắt đầu huấn luyện mô hình InceptionV3...")
inception_history = inception_model.fit(
    train_ds,
    epochs=30,  # Tăng số epochs
    validation_data=valid_ds,
    callbacks=[learning_rate_reduction_inception, early_stopping],
    verbose=1
)

In [ ]:
def plot_auc_loss(history, epochs):
    # Lấy dữ liệu từ history
    tloss = history.history['loss']  # Training loss
    vloss = history.history['val_loss']  # Validation loss

    # Kiểm tra và in ra số lượng phần tử trong các danh sách
    print(f"Number of epochs: {epochs}")
    print(f"Training loss data points: {len(tloss)}")
    print(f"Validation loss data points: {len(vloss)}")

    # Kiểm tra rằng số phần tử trong các list khớp với số epoch
    if len(tloss) != epochs or len(vloss) != epochs:
        print("Mismatch in number of epochs and data points. Adjusting to match.")
        epochs = min(len(tloss), len(vloss))  # Điều chỉnh lại epochs nếu có sự khác biệt

    Epochs = range(1, epochs + 1)  # Tạo list Epochs có độ dài giống với loss

    plt.style.use('fivethirtyeight')
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

    # Vẽ biểu đồ Training Loss và Validation Loss
    axes[0].plot(Epochs, tloss[:epochs], 'r', label='Training loss')  # Cắt phần tử phù hợp với số epoch
    axes[0].plot(Epochs, vloss[:epochs], 'g', label='Validation loss')  # Cắt phần tử phù hợp với số epoch
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()

    # Vẽ biểu đồ Accuracy nếu có
    if 'accuracy' in history.history:
        tacc = history.history['accuracy']
        vacc = history.history['val_accuracy']
        axes[1].plot(Epochs, tacc[:epochs], 'r', label='Training accuracy')  # Cắt phần tử phù hợp với số epoch
        axes[1].plot(Epochs, vacc[:epochs], 'g', label='Validation accuracy')  # Cắt phần tử phù hợp với số epoch
        axes[1].set_title('Training and Validation accuracy')
        axes[1].set_xlabel('Epochs')
        axes[1].set_ylabel('accuracy')
        axes[1].legend()

    plt.show()

# Sử dụng hàm với số epochs bạn đã dùng để huấn luyện
plot_auc_loss(history, epochs=5)


In [ ]:
def predictor(test_ds):
    y_pred, error_list, error_pred_list = [], [], []
    y_true = test_ds.labels
    classes = list(test_ds.class_indices.keys())
    class_count = len(classes)
    errors = 0
    preds = tf.argmax(net.predict(test_ds), axis=1)
    tests = len(preds)
    for i in range(tests):
        pred_index = preds[i]
        true_index = test_ds.labels[i]
        if pred_index != true_index:
            errors += 1
            file = test_ds.filenames[i]
            error_list.append(file)
            error_classes = classes[pred_index]
            error_pred_list.append(error_classes)
        y_pred.append(pred_index)

    acc = (1 - errors / tests) * 100
    msg = f'there were {errors} errors in {tests} tests for an accuracy of {acc:6.2f}%'
    print(msg)
    ypred = np.array(y_pred)
    ytrue = np.array(y_true)
    f1score = f1_score(ytrue, ypred, average='weighted') * 100

    if class_count <= 30:
        cm = confusion_matrix(y_true, y_pred)

        plt.figure(figsize=(12, 8))
        sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)
        plt.xticks(np.arange(class_count) + .5, classes, rotation=90)
        plt.yticks(np.arange(class_count) + .5, classes, rotation=0)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        plt.show()

    clr = classification_report(y_true, y_pred, target_names=classes, digits=4)
    print("Classification Report:\n----------------------\n", clr)

    return errors, tests, error_list, error_pred_list, f1score

errors, tests, error_list, error_pred_list, f1score = predictor(test_ds)

In [ ]:
import matplotlib.pyplot as plt

# Kết hợp tất cả các DataFrame train, test, và validation
all_data = pd.concat([train_df, test_df, valid_df])

# Đếm số lượng mẫu cho mỗi lớp trong toàn bộ bộ dữ liệu
class_counts_all = all_data['labels'].value_counts()

# Vẽ biểu đồ
plt.figure(figsize=(10, 6))
class_counts_all.plot(kind='bar', color='skyblue')
plt.title('Number of Samples per Class (All datasets)')
plt.xlabel('Classes')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Lưu model dưới dạng SavedModel
saved_model_path = './model/vgg19_model'
vgg19_model.save(saved_model_path, save_format='tf')
print(f"Model đã được lưu dưới dạng SavedModel tại: {saved_model_path}")

# Lưu model dưới dạng TFLite
tflite_save_path = './model/vgg19_model.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(vgg19_model)
tflite_model = converter.convert()

with open(tflite_save_path, 'wb') as f:
    f.write(tflite_model)
print(f"Model đã được lưu dưới dạng TFLite tại: {tflite_save_path}")

In [ ]:
# Tải lại model đã lưu dưới dạng SavedModel
loaded_model = tf.keras.models.load_model('./model/vgg19_model')

# Kiểm tra cấu trúc model
loaded_model.summary()

# Chạy thử dự đoán trên một batch từ valid_ds
for batch in valid_ds:
    images, labels = batch
    predictions = loaded_model.predict(images)
    print("Predictions:", predictions)
    print("True Labels:", labels)
    break  # Chỉ chạy thử trên một batch

In [ ]:
import numpy as np
import tensorflow as tf

with tf.device('/CPU:0'):
    # Đảm bảo input_size và class_names đã được định nghĩa
    input_size = (300, 300)  # Ví dụ với kích thước 600x600, thay đổi tùy theo mô hình của bạn
    class_names = ['Black Rot', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']  # Thay danh sách tên lớp vào đây

    # Sử dụng mô hình SavedModel
    loaded_model = tf.keras.models.load_model('./model/vgg19_model/')

    # Dự đoán
    def predict_with_saved_model(image_path):
        # Tải ảnh và xử lý
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=input_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        #img_array = img_array / 255.0  # Chuẩn hóa giá trị ảnh
        img_array = np.expand_dims(img_array, axis=0)  # Thêm batch dimension
        
        # Dự đoán
        predictions = loaded_model.predict(img_array)
        predicted_class = class_names[np.argmax(predictions[0])]  # Lớp có xác suất cao nhất
        
        # Trả về lớp dự đoán và xác suất dự đoán
        return predicted_class, predictions[0]

    # Ví dụ sử dụng hàm dự đoán
    image_path = 'D:/pow.jfif'
    predicted_class, prediction_probs = predict_with_saved_model(image_path)

    print(f"Predicted Class: {predicted_class}")
    print("Prediction Probabilities:")
    for class_name, prob in zip(class_names, prediction_probs):
        print(f"{class_name}: {prob:.4f}")


In [ ]:
import numpy as np
import tensorflow as tf

with tf.device('/CPU:0'):
    # Đảm bảo input_size và class_names đã được định nghĩa
    input_size = (300, 300)  # Ví dụ với kích thước 600x600, thay đổi tùy theo mô hình của bạn
    class_names = ['Black Rot', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']  # Thay danh sách tên lớp vào đây

    # Sử dụng mô hình SavedModel
    loaded_model = tf.keras.models.load_model('./model/vgg19_model/')

    # Dự đoán
    def predict_with_saved_model(image_path):
        # Tải ảnh và xử lý
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=input_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        #img_array = img_array / 255.0  # Chuẩn hóa giá trị ảnh
        img_array = np.expand_dims(img_array, axis=0)  # Thêm batch dimension
        
        # Dự đoán
        predictions = loaded_model.predict(img_array)
        predicted_class = class_names[np.argmax(predictions[0])]  # Lớp có xác suất cao nhất
        
        # Trả về lớp dự đoán và xác suất dự đoán
        return predicted_class, predictions[0]

    # Ví dụ sử dụng hàm dự đoán
    image_path = 'D:/healthy.JPG'
    predicted_class, prediction_probs = predict_with_saved_model(image_path)

    print(f"Predicted Class: {predicted_class}")
    print("Prediction Probabilities:")
    for class_name, prob in zip(class_names, prediction_probs):
        print(f"{class_name}: {prob:.4f}")


In [ ]:
import numpy as np
import tensorflow as tf

with tf.device('/CPU:0'):
    # Đảm bảo input_size và class_names đã được định nghĩa
    input_size = (300, 300)  # Ví dụ với kích thước 600x600, thay đổi tùy theo mô hình của bạn
    class_names = ['Black Rot', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']  # Thay danh sách tên lớp vào đây

    # Sử dụng mô hình SavedModel
    loaded_model = tf.keras.models.load_model('./model/vgg19_model/')

    # Dự đoán
    def predict_with_saved_model(image_path):
        # Tải ảnh và xử lý
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=input_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        #img_array = img_array / 255.0  # Chuẩn hóa giá trị ảnh
        img_array = np.expand_dims(img_array, axis=0)  # Thêm batch dimension
        
        # Dự đoán
        predictions = loaded_model.predict(img_array)
        predicted_class = class_names[np.argmax(predictions[0])]  # Lớp có xác suất cao nhất
        
        # Trả về lớp dự đoán và xác suất dự đoán
        return predicted_class, predictions[0]

    # Ví dụ sử dụng hàm dự đoán
    image_path = 'D:/risat.jpg'
    predicted_class, prediction_probs = predict_with_saved_model(image_path)

    print(f"Predicted Class: {predicted_class}")
    print("Prediction Probabilities:")
    for class_name, prob in zip(class_names, prediction_probs):
        print(f"{class_name}: {prob:.4f}")


In [ ]:
import subprocess

# Đường dẫn đến SavedModel đã lưu
saved_model_path = './model/vgg19_model'

# Đường dẫn thư mục để lưu model TensorFlow.js
tfjs_model_dir = './model/modeljs'

# Chuyển đổi model sang TensorFlow.js
result = subprocess.run([
    'tensorflowjs_converter',
    '--input_format', 'tf_saved_model',
    '--output_format', 'tfjs_graph_model',
    '--saved_model_tags', 'serve',
    saved_model_path,
    tfjs_model_dir
], capture_output=True, text=True)

if result.returncode != 0:
    print(f"Lỗi trong quá trình chuyển đổi: {result.stderr}")
else:
    print(f"Model đã được chuyển đổi sang TensorFlow.js và lưu tại: {tfjs_model_dir}")

In [ ]:
# Tải lại model và kiểm tra các trọng số trong lớp cuối
import numpy as np
import tensorflow as tf

with tf.device('/CPU:0'):
    model = tf.keras.models.load_model('./model/vgg19_model')

    # Lấy ra tầng cuối cùng
    final_layers = [layer for layer in model.layers if 'dense' in layer.name.lower() or 'res_mlp' in layer.name.lower()]
    for layer in final_layers[-2:]:  # Xem 2 tầng cuối
        weights = layer.get_weights()
        print(f"Layer: {layer.name}, Shape: {[w.shape for w in weights]}")
        # Kiểm tra bias có bị lệch không
        if len(weights) > 1:  # Có bias
            bias = weights[1]
            print(f"Bias values: {bias}")
            print(f"Bias min: {np.min(bias)}, max: {np.max(bias)}, mean: {np.mean(bias)}")